In [ ]:
from IPython.display import clear_output
import os

os.environ["CUDA_VISIBLE_DEVICES"] = "-1"
import re
import dropbox
import sys
import pandas as pd
import numpy as np
import imageio.v2 as imageio
import matplotlib.pyplot as plt
import cv2
import logging
import datetime
import glob
import json
import scipy
import matplotlib as mpl

from subprocess import call
from tifffile import imwrite
from tqdm import tqdm
from pathlib import Path
from amftrack.util.dbx import (
    upload_folder,
    download,
    read_saved_dropbox_state,
    save_dropbox_state,
    load_dbx,
    get_dropbox_folders_prince,
    get_dropbox_video_folders,
    download_video_folders_drop,
    download_analysis_folders_drop,
)
from amftrack.pipeline.launching.run import (
    run_transfer,
)
from amftrack.pipeline.launching.run_super import run_parallel_transfer
from amftrack.pipeline.launching.run_super import run_parallel_flows
from amftrack.pipeline.functions.transport_processing.high_mag_videos.plot_data import (
    plot_summary,
    save_raw_data,
)
from amftrack.pipeline.functions.transport_processing.high_mag_videos.high_mag_analysis import (
    HighmagDataset,
    VideoDataset,
    EdgeDataset,
    index_videos_dropbox_new,
    analysis_run,
)
from amftrack.pipeline.functions.transport_processing.high_mag_videos.kymo_class import (
    KymoVideoAnalysis,
    KymoEdgeAnalysis,
)
from amftrack.util.dbx import (upload
)
%load_ext autoreload
%autoreload 2
%matplotlib widget

from amftrack.pipeline.launching.run_super import (
    run_launcher,
    directory_scratch,
    directory_project,
    directory_project,
    run_parallel_stitch,
    run_parallel_transfer,
)
import os
from amftrack.pipeline.functions.image_processing.experiment_class_surf import (
    Experiment,
    save_graphs,
    load_graphs,
    Edge,
    Node,

)
from amftrack.pipeline.functions.image_processing.experiment_util import (
    get_random_edge,
    distance_point_edge,
    plot_edge,
    plot_edge_cropped,
    find_nearest_edge,
    get_edge_from_node_labels,
    plot_full_image_with_features,
    get_all_edges,
    get_all_nodes,
    find_neighboring_edges,
    reconstruct_image,
    reconstruct_skeletton_from_edges,
    reconstruct_skeletton_unicolor,
    reconstruct_image_from_general,
    plot_full,
    plot_edge_color_value,
)
from amftrack.transport.align_video_network import identify_nodes
logging.basicConfig(stream=sys.stdout, level=logging.debug)
from amftrack.util.sys import (
    get_dates_datetime,
    get_dirname,
    temp_path,
    get_data_info,
    update_plate_info,
    update_analysis_info,
    get_analysis_info,
    get_current_folders,
    get_folders_by_plate_id,
)
mpl.rcParams["figure.dpi"] = 100
from amftrack.pipeline.functions.transport_processing.high_mag_videos.register_videos import *

In [ ]:
plate_id = "310_20230830"
plate_id_video = "20230904_Plate310"
videos_folder = "/projects/0/einf914/videos/"
plate_id = "441_20230807"
plate_id_video = "20230811_Plate441"
videos_folder = "/projects/0/einf914/videos/"

In [ ]:
indexes = {
"20230901_Plate310" : 20,
"20230902_Plate310" : 33,
"20230903_Plate310" : 42,
"20230904_Plate310" : 52,
"20230905_Plate310" : 64,
"20230906_Plate310" : 73,
}
indexes = {
"20230810_Plate441" : 29,
"20230811_Plate441" : 41,
"20230812_Plate441" : 47,
"20230813_Plate441" : 60,
}

In [ ]:
analysis_folder = "/projects/0/einf914/analysis_videos/CocoTransport/"
analysis_folder = f"{analysis_folder}{plate_id_video}/"

img_infos = glob.glob(f"{analysis_folder}/**/video_data.json", recursive=True)
vid_anls_frame = pd.DataFrame()
add_infos = []
for address in img_infos:
    add_infos.append(pd.read_json(address, orient="index").T)
vid_anls_frame = pd.concat([vid_anls_frame] + add_infos, ignore_index=True)

vid_anls_frame = vid_anls_frame.sort_values("unique_id").reset_index(drop=True)
vid_anls_frame_select = vid_anls_frame.loc[vid_anls_frame["plate_id"] == plate_id_video]
columns_to_drop = ['xpos_network', 'ypos_network']

# Dropping columns from vid_anls_frame_select_network if they exist
for column in columns_to_drop:
    if column in vid_anls_frame_select.columns:
        vid_anls_frame_select = vid_anls_frame_select.drop(column, axis=1)


In [ ]:
analysis_folder = "/projects/0/einf914/analysis_videos/CocoTransport/"
analysis_folder = f"{analysis_folder}{plate_id_video}/"

img_infos = glob.glob(f"{analysis_folder}/**/video_data_network.json", recursive=True)
vid_anls_frame = pd.DataFrame()
add_infos = []
for address in img_infos:
    add_infos.append(pd.read_json(address, orient="index").T)
vid_anls_frame = pd.concat([vid_anls_frame] + add_infos, ignore_index=True)

vid_anls_frame = vid_anls_frame.sort_values("unique_id").reset_index(drop=True)
vid_anls_frame_select_network = vid_anls_frame.loc[vid_anls_frame["plate_id"] == plate_id_video]

In [ ]:
vid_anls_frame_merged = vid_anls_frame_select.merge(vid_anls_frame_select_network[['xpos_network','ypos_network','unique_id']],on = 'unique_id')
# vid_anls_frame_merged = vid_anls_frame_select

In [ ]:
# vid_anls_frame_merged['folder'] = vid_anls_frame_merged['folder'].str.replace('/Img','')

In [ ]:
analysis_folder_root = "/projects/0/einf914/analysis_videos/"


In [ ]:
vid_anls_frame_merged["analysis_folder"] = analysis_folder_root
vid_anls_frame_merged["videos_folder"] = [
    str(Path(videos_folder) / entry["folder"])
    for index, entry in vid_anls_frame_merged.iterrows()
]

In [ ]:
data_obj = HighmagDataset(vid_anls_frame_merged, analysis_folder_root, videos_folder)

In [ ]:
# directory_targ = os.path.join(directory_scratch, "stitch_temp2") + "/"
directory_targ = '/projects/0/einf914/transport/'
update_plate_info(directory_targ, local=True)
all_folders = get_current_folders(directory_targ, local=True)

In [ ]:
all_folders['unique_id'].unique()

In [ ]:
folders = all_folders.loc[all_folders["unique_id"] == plate_id]
folders = folders.loc[folders["/Analysis/nx_graph_pruned_labeled.p"] == True]

In [ ]:
folders = folders.sort_values(by="datetime")

exp = Experiment(directory_targ)
i = 41
exp.load(folders.iloc[i : i + 1], suffix="_width")
for t in range(exp.ts):
    exp.load_tile_information(t)

In [ ]:
data_obj.video_objs = sorted(data_obj.video_objs,key = lambda video : video.dataset['video_int'])

In [ ]:
mpl.rcParams["figure.dpi"] = 50

from random import choice
vid_obj = choice(data_obj.video_objs)
vid_obj = data_obj.video_objs[34]

vid_obj.plot_speed_arrows(plot_both=True, video_txt_size=30)


In [ ]:
Rcurrent,tcurrent = np.array([[1,0],[0,1]]), np.array([0 ,0])   

positions = Rcurrent@np.array(vid_obj.dataset[["xpos_network", "ypos_network"]])+tcurrent
positions_list = [positions.tolist()]
window = np.array([100,100])
begin= (positions-window).astype(int)
end = (positions+window).astype(int)
region=[[begin[0],begin[1]],[end[0],end[1]]]
# region = [[100, 100], [2000,2000]]

In [ ]:
region

In [ ]:
shiftx = vid_obj.img_dim[0]*vid_obj.space_res/1.725/2
shifty = vid_obj.img_dim[1]*vid_obj.space_res/1.725/2

In [ ]:
segments = get_segments_ends(vid_obj,shiftx,shifty,20,Rcurrent,tcurrent)


In [ ]:
edges = get_all_edges(exp, t)

edges = [edge for edge in edges if dist_edge(edge,positions,t)<=100]
pixels = [pixel for edge in edges for pixel in edge.pixel_list(t)]
pixels = [pixel for pixel in pixels if np.linalg.norm(pixel-positions)<=150]    

In [ ]:
segment_points = []
for begin, end in segments:
    # Include the start point, interpolated points, and the end point
    interpolated_points = interpolate_points(begin, end)
    segment_points.extend(interpolated_points)
    segment_points.append(end)  # Ensure the end point is included

segment_points = np.array(segment_points)


In [ ]:
from amftrack.pipeline.functions.transport_processing.high_mag_videos.plotting import *

positions_list = [positions.tolist()]
# plt.close("all")
t = 0
vmax = 9
vmin = 3
nodes = get_all_nodes(exp, t)
edges = get_all_edges(exp, t)
edges = [edge for edge in edges if dist_edge(edge,positions,t)<=100]
downsizing = 1

plot_full_video(
    exp,
    t,
    downsizing=downsizing,
    points=positions_list,
    video_num=[0],
    edges=edges,
    dilation=5,
    region = region,
    segments = segments
)

In [ ]:
fig,ax = plt.subplots()

# Plotting point cloud
px, py = zip(*pixels)  # Unpacking points into x and y coordinates
ax.scatter(px, py, color='blue')  # Plot points in blue
px, py = zip(*segment_points)  # Unpacking points into x and y coordinates
ax.scatter(px, py, color='red')  # Plot points in blue

In [ ]:
Y = np.array(pixels)
X = np.array(segment_points)

In [ ]:
import probreg as pr
# X = np.insert(X, 2, values=0, axis=1)  # Inserting a Z-axis with zero values
# Y = np.insert(Y, 2, values=0, axis=1)  # Inserting a Z-axis with zero values

# Convert numpy arrays to point clouds.
# In probreg, point clouds can be represented directly as numpy arrays.
source = X
target = Y
# reg = cpd.registration(target)


In [ ]:
Rcurrent,tcurrent = find_optimal_R_and_t(source, target)

In [ ]:
Rcurrent,tcurrent

In [ ]:
transformation_matrix = np.eye(4)
transformation_matrix[:3, :3] = cpd.transformation.rot  # Rotation
transformation_matrix[:3, 3] = cpd.transformation.t  # Translation

In [ ]:
# Transform the source point cloud using the obtained transformation matrix
transformed_source = np.dot(R, source.T).T + t

# Calculate distances between each pair of corresponding points
distances = average_min_distance_to_set(transformed_source, target)

# Compute the root mean square deviation
rmsd = np.sqrt(np.mean(distances**2))

rmsd

In [ ]:
Rcurrent,tcurrent = cpd.transformation.rot[:2,:2],cpd.transformation.t[:2]

In [ ]:
Rcurrent,tcurrent = np.array([[1,0],[0,1]]), np.array([0 ,0])   
mapping, dist,Rfound,tfound = make_whole_mapping(vid_obj,exp,t,dist = 100,R=Rcurrent,trans = tcurrent)
if np.linalg.det(Rfound)>0:
    Rcurrent = Rfound@Rcurrent
    tcurrent = Rfound@tcurrent+tfound
    if dist>20:
        mapping, dist,Rfound,tfound = make_whole_mapping(vid_obj,exp,t,dist = 100,R=Rcurrent,trans = tcurrent)
        if np.linalg.det(Rfound)>0:
            Rcurrent = Rfound@Rcurrent
            tcurrent = Rfound@tcurrent+tfound

In [ ]:
np.linalg.det(Rfound)

In [ ]:
for edge in vid_obj.edge_objs:

    edge_data_csv.loc[edge_data_csv['edge_name'] == edge.edge_name, 'width'] = 2
edge_data_csv

In [ ]:
def add_attribute(edge_data_csv,vid_edge_obj,network_edge_attribute,name_new_col,mapping):
    new_attribute = network_edge_attribute(mapping[vid_edge_obj.edge_name])   
    # print(new_attribute)
    edge_data_csv.loc[edge_data_csv['edge_name'] == vid_edge_obj.edge_name, name_new_col] = new_attribute


In [ ]:
for edge in vid_obj.edge_objs:
    add_attribute(edge_data_csv,edge,lambda edge: edge.width(t),"width",mapping)
    add_attribute(edge_data_csv,edge,lambda edge: edge.end.label,"network_end",mapping)
    add_attribute(edge_data_csv,edge,lambda edge: edge.begin.label,"network_begin",mapping) 
    add_attribute(edge_data_csv,edge,lambda edge : dist,"mapping_quality",mapping)            

In [ ]:
edge_data_csv

In [ ]:
import numpy as np
import pandas as pd
def check_hasedges(vid_obj):
    shiftx = vid_obj.img_dim[0]*vid_obj.space_res/1.725/2
    shifty = vid_obj.img_dim[1]*vid_obj.space_res/1.725/2
    segments = get_segments_ends(vid_obj,shiftx,shifty,40)
    return(len(segments)>0)

def initialize_transformation():
    return np.array([[1,0],[0,1]]), np.array([0,0])

def update_transformation(Rcurrent, tcurrent, Rfound, tfound):
    Rcurrent = Rfound @ Rcurrent
    tcurrent = Rfound @ tcurrent + tfound
    return Rcurrent, tcurrent

def should_reset(Rfound):
    return np.linalg.det(Rfound) <= 0 or Rfound[0][0] <= 0.99

def attempt_mapping(vid_obj, exp, t, Rcurrent, tcurrent):
    try:
        mapping, dist, Rfound, tfound = make_whole_mapping(vid_obj, exp, t, dist=100, R=Rcurrent, trans=tcurrent)
    except IndexError:
        Rcurrent, tcurrent = initialize_transformation()
        mapping, dist, Rfound, tfound = make_whole_mapping(vid_obj, exp, t, dist=100, R=Rcurrent, trans=tcurrent)
    return mapping, dist, Rfound, tfound

def process_video_object(vid_obj, exp, t, Rcurrent, tcurrent):
    mapping, dist, Rfound, tfound = attempt_mapping(vid_obj, exp, t, Rcurrent, tcurrent)
    if np.linalg.det(Rfound) > 0 and Rfound[0][0] > 0.99:
        Rcurrent, tcurrent = update_transformation(Rcurrent, tcurrent, Rfound, tfound)
        if dist > 20:
            mapping, dist, Rfound, tfound = attempt_mapping(vid_obj, exp, t, Rcurrent, tcurrent)
            if should_reset(Rfound):
                Rcurrent, tcurrent = initialize_transformation()
    else:
        Rcurrent, tcurrent = initialize_transformation()
    return Rcurrent, tcurrent, mapping, dist

def register_dataset(data_obj, exp, t):
    Rcurrent, tcurrent = initialize_transformation()

    for index, vid_obj in enumerate(data_obj.video_objs[35:]):
        if check_hasedges(vid_obj):
            Rcurrent, tcurrent, mapping, dist = process_video_object(vid_obj, exp, t, Rcurrent, tcurrent)
            print(index, dist, Rcurrent, tcurrent)
            update_edge_attributes(vid_obj, mapping, dist, t)

def update_edge_attributes(vid_obj, mapping, dist, t):
    edge_data_csv = pd.read_csv(vid_obj.edge_adr)
    for edge in vid_obj.edge_objs:
        add_attribute(edge_data_csv, edge, lambda edge: edge.width(t), "width", mapping)
        add_attribute(edge_data_csv, edge, lambda edge: edge.end.label, "network_end", mapping)
        add_attribute(edge_data_csv, edge, lambda edge: edge.begin.label, "network_begin", mapping)
        add_attribute(edge_data_csv, edge, lambda edge: dist, "mapping_quality", mapping)
    edge_data_csv.to_csv(vid_obj.edge_adr)


In [ ]:
register_dataset(data_obj,exp,t)